In [5]:
import pandas as pd

In [6]:
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP(r"C:\Users\anhdq33\Downloads\VinBigData\ML\Project\2020_VBDI_BML\VnCoreNLP-master\VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [7]:
df = pd.read_csv('./data/data_clean_v4.csv').drop('Unnamed: 0', axis=1)
df.head()

,index,NewsId,Ticker,NewsTitle,NewsFullContent,PublicDate,len,word_len
0,311,1441,VCB,VCB ký hợp đồng vay vốn 100 triệu USD,ông nguyễn hòa bình chủ tịch hđqt của vcb cho ...,2008-10-20 00:00:00,1054,8
1,768,4086,STB,"Kết thúc Phiên 1, 30/10: STB và PPC đang &quot...",trong khi thị trường châu âu có phiên tăng điể...,2008-10-30 10:08:00,2550,8
2,4639,19833,PNJ,"Ngày 23/3, Hose đón tân binh PNJ",kế hoạch này đã được đại diện pnj bà cao thị n...,2009-03-03 11:29:00,1941,8
3,10448,25591,FPT,Sắp khởi công Đại học FPT,theo đó tháng 7/2009 đại học fpt sẽ chính thức...,2009-06-02 14:29:00,1255,6
4,10879,26019,SSI,SSI đứng đầu thị phần môi giới cổ phiếu,trong khi chỉ có 5 công ty nắm tới trên 30 ngh...,2009-06-09 17:13:00,1158,6


In [8]:
dic = pd.read_csv('./data/fullTranslated.csv')['words'].values
len(dic)

23534

In [9]:
def word_segment(text):
    annotated_text = annotator.annotate(text)['sentences'][0]
    return [dic['form'].replace('_',' ') for dic in annotated_text] 

In [10]:
df['word_segment'] = df['NewsFullContent'].apply(lambda text: word_segment(text))

In [11]:
def tag(word, dic):
    if word in dic:
        return 1
    else:
        return 0

def label(text, dic):
    keywords = set(text).intersection(dic)
    text_df = pd.DataFrame(text)
    text_df.columns = ['word']

    text_df['tag'] = text_df['word'].apply(lambda x: tag(x, keywords))
    
#     return list(zip(text_df['word'], text_df['tag']))
    return sum(text_df['tag']), list(zip(text_df['word'], text_df['tag']))

In [12]:
df['filtered'] = df['word_segment'].apply(lambda text: label(text, dic))

In [13]:
df['match'], df['label'] = zip(*df.filtered.values)

C:\Users\anhdq33\Anaconda3\envs\keyword_extract\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [14]:
df[df['match'] > 30][['NewsTitle','NewsFullContent','label']].to_csv('./data/data_labeled.csv')

In [15]:
df[df['match'] > 30][['NewsTitle','NewsFullContent','label']]

,NewsTitle,NewsFullContent,label
0,VCB ký hợp đồng vay vốn 100 triệu USD,ông nguyễn hòa bình chủ tịch hđqt của vcb cho ...,"[(ông, 0), (nguyễn hoà bình, 0), (chủ tịch, 1)..."
1,"Kết thúc Phiên 1, 30/10: STB và PPC đang &quot...",trong khi thị trường châu âu có phiên tăng điể...,"[(trong, 0), (khi, 0), (thị trường, 1), (châu ..."
2,"Ngày 23/3, Hose đón tân binh PNJ",kế hoạch này đã được đại diện pnj bà cao thị n...,"[(kế hoạch, 1), (này, 0), (đã, 0), (được, 1), ..."
3,Sắp khởi công Đại học FPT,theo đó tháng 7/2009 đại học fpt sẽ chính thức...,"[(theo, 1), (đó, 0), (tháng, 1), (7/2009, 0), ..."
4,SSI đứng đầu thị phần môi giới cổ phiếu,trong khi chỉ có 5 công ty nắm tới trên 30 ngh...,"[(trong, 0), (khi, 0), (chỉ, 1), (có, 1), (5, ..."
...,...,...,...
925,Liên danh FPT Quy Nhơn trúng thầu dự án hơn 4....,ông nguyễn phi long pct ubnd tỉnh bình định vừ...,"[(ông, 0), (nguyễn phi, 0), (long, 0), (pct, 0..."
926,Liên danh FPT Quy Nhơn trúng thầu dự án hơn 4....,ông nguyễn phi long pct ubnd tỉnh bình định vừ...,"[(ông, 0), (nguyễn phi, 0), (long, 0), (pct, 0..."
927,"PNJ phát hành 2,3 triệu cổ phiếu ESOP, Chủ tịc...",giá phát hành dự kiến là 20 000 đồng/cp thấp...,"[(giá, 1), (phát hành, 1), (dự kiến, 1), (là, ..."
928,"Vingroup nêu phương án quy hoạch khu đô thị, s...",đại diện ubnd bắc giang đóng góp ý kiến thiế...,"[(đại diện, 1), (ubnd, 0), (bắc giang, 0), (đó..."


In [16]:
ner = VnCoreNLP("./VnCoreNLP-master/VnCoreNLP-1.1.1.jar", annotators="wseg,pos,ner", max_heap_size='-Xmx2g') 

In [17]:
trial = ner.annotate("Hà Nội là thủ đô của Việt Nam")

In [18]:
trial['sentences'][0]

[{'index': 1,
  'form': 'Hà_Nội',
  'posTag': 'Np',
  'nerLabel': 'B-LOC',
  'head': -1},
 {'index': 2, 'form': 'là', 'posTag': 'V', 'nerLabel': 'O', 'head': -1},
 {'index': 3, 'form': 'thủ_đô', 'posTag': 'N', 'nerLabel': 'O', 'head': -1},
 {'index': 4, 'form': 'của', 'posTag': 'E', 'nerLabel': 'O', 'head': -1},
 {'index': 5,
  'form': 'Việt_Nam',
  'posTag': 'Np',
  'nerLabel': 'B-LOC',
  'head': -1}]

In [19]:
df2 = pd.read_csv('./data/data_clean_v3.csv').drop('Unnamed: 0', axis=1)
df2.head()

,index,NewsId,Ticker,NewsTitle,NewsFullContent,PublicDate,len,word_len
0,0,1441,VCB,VCB ký hợp đồng vay vốn 100 triệu USD,Ông Nguyễn Hòa Bình Chủ tịch HĐQT của VCB cho ...,2008-10-20 00:00:00,1054,8
1,1,4086,STB,"Kết thúc Phiên 1, 30/10: STB và PPC đang &quot...",Trong khi thị trường Châu Âu có phiên tăng điể...,2008-10-30 10:08:00,2550,8
2,2,19833,PNJ,"Ngày 23/3, Hose đón tân binh PNJ",Kế hoạch này đã được đại diện PNJ bà Cao Thị N...,2009-03-03 11:29:00,1941,8
3,3,25591,FPT,Sắp khởi công Đại học FPT,Theo đó tháng 7/2009 Đại học FPT sẽ chính thức...,2009-06-02 14:29:00,1255,6
4,4,26019,SSI,SSI đứng đầu thị phần môi giới cổ phiếu,Trong khi chỉ có 5 công ty nắm tới trên 30 ngh...,2009-06-09 17:13:00,1158,6


In [20]:
trial = ner.annotate(df2.loc[0,'NewsFullContent'])

In [21]:
def vnner(text): # part-of-speech tagging
    annotated_text = ner.annotate(text)
    pos = []
    for dic in annotated_text['sentences'][0]:
#         if dic['posTag'] not in ['L','M','E','C','Cc','I','T','Y','Z','X','CH']:
          pos.append(tuple([dic['form'], dic['nerLabel']]))
        
    return pos

In [22]:
vnner(df2.loc[0,'NewsFullContent'])

[('Ông', 'O'),
 ('Nguyễn_Hoà_Bình_Chủ_tịch', 'O'),
 ('HĐQT', 'O'),
 ('của', 'O'),
 ('VCB', 'B-ORG'),
 ('cho', 'O'),
 ('biết', 'O'),
 ('khoản', 'O'),
 ('vay', 'O'),
 ('trong', 'O'),
 ('3', 'O'),
 ('năm', 'O'),
 ('này', 'O'),
 ('sẽ', 'O'),
 ('góp_phần', 'O'),
 ('bổ_sung', 'O'),
 ('nguồn', 'O'),
 ('vốn', 'O'),
 ('ngoại_tệ', 'O'),
 ('trung_hạn', 'O'),
 ('đáng_kể', 'O'),
 ('phục_vụ', 'O'),
 ('cho', 'O'),
 ('các', 'O'),
 ('mục_tiêu', 'O'),
 ('kinh_doanh', 'O'),
 ('và', 'O'),
 ('tăng_trưởng', 'O'),
 ('của', 'O'),
 ('ngân_hàng', 'O'),
 ('Hiện_tại', 'O'),
 ('tổng', 'O'),
 ('nguồn', 'O'),
 ('vốn', 'O'),
 ('của', 'O'),
 ('VCB', 'O'),
 ('đạt', 'O'),
 ('203', 'O'),
 ('ngàn', 'O'),
 ('tỷ', 'O'),
 ('đồng', 'O'),
 ('tăng', 'O'),
 ('6', 'O'),
 ('000', 'O'),
 ('tỷ', 'O'),
 ('đồng', 'O'),
 ('so', 'O'),
 ('với', 'O'),
 ('cuối', 'O'),
 ('năm', 'O'),
 ('2007', 'O'),
 (';', 'O'),
 ('trong', 'O'),
 ('đó', 'O'),
 ('nguồn', 'O'),
 ('vốn', 'O'),
 ('ngoại_tệ', 'O'),
 ('là', 'O'),
 ('5', 'O'),
 ('3', 'O'),
 ('tỷ',